# Cell Operation with pET19K_HF_alpha 

## 1. Load the python packages

In [1]:
from __future__ import print_function, division, absolute_import

# python imports
import re
from os.path import join
from collections import defaultdict
import pickle
import pandas as pd

# third party imports
import pandas
import tabulate
import cobra
pd.set_option('display.max_columns', 100)
pd.set_option('display.width',100)
pd.set_option('display.max_colwidth',300)

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
#from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/solvers/__init__.py:63: UserWarning: No LP solvers found
  warn("No LP solvers found")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/usr/lib/python3.6/site-packages/cobra-0.5.11-py3.6-linux-x86_64.egg/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


## 2. Load the iJL1678b model

Load the iJL1678b model in pickle format that is located in the current directory.

In [2]:
with open('iJL1678b.pickle', 'rb') as f:
    me = pickle.load(f)

## 3. Define ME-Model Solve Functions

In [3]:
def solve_me_model(me, max_mu, precision=1e-6, min_mu=0, using_soplex=False,
                  compiled_expressions=None):
    if using_soplex:
        from cobrame.solve.algorithms import binary_search
        binary_search(me, min_mu=min_mu, max_mu=max_mu, debug=True, mu_accuracy=precision,
                      compiled_expressions=compiled_expressions)
    else:
        from qminospy.me1 import ME_NLP1
        # The object containing solveME methods--composite that uses a ME model object 
        me_nlp = ME_NLP1(me, growth_key='mu')
        # Use bisection for now (until the NLP formulation is worked out)
        muopt, hs, xopt, cache = me_nlp.bisectmu(precision=precision, mumax=max_mu)
        me.solution.f = me.solution.x_dict['biomass_dilution']
        
def show_escher_map(me, solution=None):
    import escher
    view = escher.Builder("iJO1366.Central metabolism")
    view.reaction_data = me.get_metabolic_flux(solution=solution)
    return view

## 4. Create the Hagfish-alpha Transcription Reaction

Complete the steps necessary to create the Alpha Transcription Reaction

In [4]:
locus_id = 'Alpha'
left_pos =  None
right_pos = None
seq = 'ATGAGCATCAGCCAGACCGTTAGCAAAAGCTATACCAAAAGCGTTAGCCGTGGTGGTCAGGGTGTTAGCTATAGCCAGAGCAGCAGCCATAAAGTTGGTGGTGGTAGCGTTCGTTATGGCACCACCTATAGCAGCGGTGGTATTAGCCGTGTTCTGGGTTTTCAAGGTGGTGCAGGCGGTGCAGCAAGCGCAGGTTTTGGTGGTAGTGTTGGTGGTTCAGGTCTGAGCCGTGTGTTAGGTGGTAGCATGGTTAGCGGTTATCGTAGCGGTATGGGTGTTGGCGGTCTGAGTCTGAGCGGCACCGCAGGTCTGCCGGTTAGCCTGCGTGGTGTTGGTGCAGGTAAAGCACTGCATGCAATTACCAGCGCATTTCGTACACGCGTTGGTGGTCCGGGTACAAGCGTTGGCGGTTATGGTGTGAATTATAGCTTTCTGCCGAGCACAGCAGGTCCGAGCTTTGGTGGCCCTTTTGGCGGTCCGTTTGGAGGTCCTTTTGGTGGACCATTAGGTCCGGGTTATATTGATCCGGCAACACTGCCGAGTCCGGATACCGTTCAGCATACCCGTATTCGTGAAAAACAGGATCTGCAGACACTGAATACCAAATTTGCCAATCTGGTTGATCAGGTTCGTACCCTGGAACAGCATAATGCAATTCTGAAAGCACAGATTAGCATGATTACCAGTCCGAGCGATACACCGGAAGGTCCGGTTAATACCGCAGTTGTTGCAAGCACCGTTACCGCAACCTATAATGCACAGATTGAAGATCTGCGTACCACCAATACCGCACTGCATAGCGAAATTGATCATCTGACAACCATCATCAACGACATCACCACCAAATATGAAGAACAGGTTGAAGTTACCCGTACACTGGAAACCGATTGGAATACCAACAAAGATAACATCGATAACACCTACCTGACCATTGTTGATCTGCAAACCAAAGTTCAAGGTCTGGATGAGCAGATTAATACCACAAAGCAGATTTATAACGCACGTGTTCGTGAAGTTCAGGCAGCAGTTACCGGTGGTCCGACCGCAGCATATAGCATTCGTGTTGATAATACCCATCAGGCCATTGATCTGACCACCTCACTGCAAGAAATGAAAACACATTATGAAGTGCTGGCAACCAAAAGCCGTGAAGAAGCATTTACCCAGGTTCAGCCTCGTATTCAAGAAATGGCAGTGACCGTGCAGGCAGGTCCGCAGGCAATTATTCAGGCAAAAGAACAAATCCACGTGTTCAAACTGCAGATTGATAGCGTTCATCGTGAAATCGATCGTCTGCATCGTAAAAATACCGATGTGGAACGTGAAATTACCGTGATCGAAACCAATATTCATACCCAGAGTGATGAATGGACCAACAATATTAACAGCCTGAAAGTTGATCTGGAAGTGATCAAAAAGCAGATCACCCAGTATGCACGTGATTATCAAGATCTGCTGGCCACAAAAATGAGCCTGGATGTTGAAATTGCAGCCTACAAAAAACTGCTGGATAGCGAAGAAACCCGTATTAGTCATGGTGGTGGCATTACCATTACAACCAATGCAGGCACCTTTCCTGGTGGTCTGAGCGCAGCCCCAGGTGGTGGTGCAAGCTATGCAATGGTTCCGGCAGGCGTTGGAGGTGTTGGTCTGGCAGGCGTAGGTGGTTATGGTTTTCGTAGCATGGGTGGCGGTGGTGGCGTTGGCTATGGTGCCGGTGGCGGAGGTGTAGGTTATGGCGTCGGTGGTGGTTTTGGAGGTGGTATGGGTATGAGCATGAGCCGTATGAGTATGGGTGCAGCAGTTGGCGGTGGCAGCTATGGTAGCGGCAGCGGTTATAGCGGTGGCTTTGGTCTGAGTAGCAGCCGTGCAGGTTATAGTGCAAGCCGTAAAAGCTATTCAAGCGCACGTAGCAGCAGTCGTATCTAT'
strand = None
rna_type = 'mRNA'

building.create_transcribed_gene(me, locus_id, rna_type, seq)
building.add_transcription_reaction(me, 'TU_Alpha', {'Alpha'}, seq)
print(me.reactions.transcription_TU_Alpha.reaction)
me.reactions.transcription_TU_Alpha.update()

469 atp_c + 549 ctp_c + 411 gtp_c + 500 utp_c --> RNA_Alpha + 614.960167029 mRNA_biomass + 1929 ppi_c


/source/cobrame/cobrame/core/reaction.py:852 UserWarning: RNA Polymerase () not found


###  Add the RNAP to the Alpha Transcription Reaction

In [5]:
me.reactions.transcription_TU_Alpha.transcription_data.RNA_polymerase = 'RNAP70-CPLX'
me.reactions.transcription_TU_Alpha.update()

List the transcription data ("transcription_data") associated with the Alpha transcription reaction ("transcription_TU_Alpha").

In [6]:
rxn = me.reactions.transcription_TU_Alpha
data = rxn.transcription_data
pd.DataFrame({i: str(v) for i, v in data.__dict__.items()}, index=['Attribute Values']).T

,Attribute Values
id,TU_Alpha
_model,iJL1678b-ME
_parent_reactions,{'transcription_TU_Alpha'}
nucleotide_sequence,ATGAGCATCAGCCAGACCGTTAGCAAAAGCTATACCAAAAGCGTTAGCCGTGGTGGTCAGGGTGTTAGCTATAGCCAGAGCAGCAGCCATAAAGTTGGTGGTGGTAGCGTTCGTTATGGCACCACCTATAGCAGCGGTGGTATTAGCCGTGTTCTGGGTTTTCAAGGTGGTGCAGGCGGTGCAGCAAGCGCAGGTTTTGGTGGTAGTGTTGGTGGTTCAGGTCTGAGCCGTGTGTTAGGTGGTAGCATGGTTAGCGGTTATCGTAGCGGTATGGGTGTTGGCGGTCTGAGTCTGAG...
RNA_products,{'RNA_Alpha'}
RNA_polymerase,RNAP70-CPLX
subreactions,"defaultdict(<class 'int'>, {})"


List the attributes of the Alpha transcription reaction.

In [7]:
rxn = me.reactions.transcription_TU_Alpha
data = rxn
pd.DataFrame({i: str(v) for i, v in data.__dict__.items()}, index=['Attribute Values']).T

,Attribute Values
id,transcription_TU_Alpha
name,
notes,{}
annotation,{}
_gene_reaction_rule,
subsystem,
_genes,set()
_metabolites,"{<RNAP RNAP70-CPLX at 0x7ff1172073c8>: -0.00921844765619026*mu - 0.00360902225739849, <TranscribedGene RNA_Alpha at 0x7ff11732fc88>: 1, <Metabolite utp_c at 0x7ff117207e48>: -500, <Metabolite atp_c at 0x7ff1172079b0>: -469, <Metabolite ctp_c at 0x7ff117207f28>: -549, <Metabolite gtp_c at 0x7ff11..."
_model,iJL1678b-ME
_objective_coefficient,0.0


List the complete reaction associated with the Alpha transcription reaction.

In [8]:
me.reactions.get_by_id('transcription_TU_Alpha').reaction

'0.00921844765619026*mu + 0.00360902225739849 RNAP70-CPLX + 469 atp_c + 549 ctp_c + 411 gtp_c + 500 utp_c --> RNA_Alpha + 614.960167029 mRNA_biomass + 1929 ppi_c'

List the RNAP associated with the Alpha transcription reaction

In [9]:
me.reactions.transcription_TU_Alpha.transcription_data.RNA_polymerase

'RNAP70-CPLX'

## 5. Create the Alpha Translation Reaction

Complete all the steps necessary to create the Alpha Translation Reaction

### Storing information about a translation reaction

In [10]:
data = cobrame.TranslationData('Alpha', me, 'RNA_Alpha', 'protein_Alpha')
data.nucleotide_sequence = seq

### Create translation reaction

By associating the TranslationReaction with its corresponding TranslationData object and running the update function, COBRAme will create a translation reaction for the nucleotide sequence given based on the organisms codon table and prespecified translation machinery.

In [11]:
rxn = cobrame.TranslationReaction('translation_Alpha')
rxn.translation_data = data
me.add_reaction(rxn)

### Updating the translation reaction

Note that this process does not include the ribosome in the reaction.

In [12]:
rxn.update()
print(rxn.reaction)

0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_Alpha + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 48 ala__L_c + 30 arg__L_c + 21 asn__L_c + 24 asp__L_c + 2.88274348422497*(mu + 0.3915)/mu atp_c + 31 gln__L_c + 26 glu__L_c + 102 gly_c + 13 his__L_c + 36 ile__L_c + 38 leu__L_c + 23 lys__L_c + 3.67795106755342*(mu + 0.3915)/mu mRNA_biomass + 14 met__L_c + 15 phe__L_c + 21 pro__L_c + 0.00921844765619026*mu + 0.00360902225739849 ribosome + 66 ser__L_c + 58 thr__L_c + 2 trp__L_c + 26 tyr__L_c + 49 val__L_c --> 2.88274348422497*(mu + 0.3915)/mu adp_c + 2.80499314128944*(mu + 0.3915)/mu amp_c + 3.28345679012346*(mu + 0.3915)/mu cmp_c + 2.45810699588477*(mu + 0.3915)/mu gmp_c + 642.0 - 2.88274348422497*(mu + 0.3915)/mu h2o_c + 2.88274348422497*(mu + 0.3915)/mu h_c + 2.88274348422497*(mu + 0.3915)/mu pi_c + protein_Alpha + 66.69648898000001 protein_biomass + 2.99039780521262*(mu + 0.3915)/mu ump_c


### Incorporate the ribosome

Add the ribosome to the reaction.

In [13]:
ribosome = cobrame.Ribosome('ribosome')
me.add_metabolites([ribosome])
me.reactions.translation_Alpha.update()
print(me.reactions.translation_Alpha.reaction)

0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_Alpha + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 48 ala__L_c + 30 arg__L_c + 21 asn__L_c + 24 asp__L_c + 2.88274348422497*(mu + 0.3915)/mu atp_c + 31 gln__L_c + 26 glu__L_c + 102 gly_c + 13 his__L_c + 36 ile__L_c + 38 leu__L_c + 23 lys__L_c + 3.67795106755342*(mu + 0.3915)/mu mRNA_biomass + 14 met__L_c + 15 phe__L_c + 21 pro__L_c + 0.00921844765619026*mu + 0.00360902225739849 ribosome + 66 ser__L_c + 58 thr__L_c + 2 trp__L_c + 26 tyr__L_c + 49 val__L_c --> 2.88274348422497*(mu + 0.3915)/mu adp_c + 2.80499314128944*(mu + 0.3915)/mu amp_c + 3.28345679012346*(mu + 0.3915)/mu cmp_c + 2.45810699588477*(mu + 0.3915)/mu gmp_c + 642.0 - 2.88274348422497*(mu + 0.3915)/mu h2o_c + 2.88274348422497*(mu + 0.3915)/mu h_c + 2.88274348422497*(mu + 0.3915)/mu pi_c + protein_Alpha + 66.69648898000001 protein_biomass + 2.99039780521262*(mu + 0.3915)/mu ump_c


### Add Subreactions to the Alpha model

Create the subreactions that represent elongation, initiation and termination

In [14]:
elongation = ['FusA_mono_elongation', 'Tuf_gtp_regeneration']
initiation = ['Translation_gtp_initiation_factor_InfB', 'Translation_initiation_factor_InfA', 
              'Translation_initiation_factor_InfC', 'fmet_addition_at_START', 'peptide_deformylase_processing', 
              'peptide_chain_release', 'ribosome_recycler']
termination = {'UGA':'PrfB_mono'}
me.process_data.get_by_id('Alpha').add_elongation_subreactions(elongation_subreactions=set(elongation))
me.process_data.get_by_id('Alpha').add_initiation_subreactions(start_codons=set(['AUG']), start_subreactions=set(initiation))
me.process_data.get_by_id('Alpha').add_termination_subreactions(translation_terminator_dict=termination)
#me.reactions.translation_Alpha.update(verbose=False)

/source/cobrame/cobrame/core/processdata.py:896 UserWarning: No termination enzyme for RNA_Alpha


List the subreactions associated with Alpha translation reaction stored in the ProcessData.

In [15]:
me.process_data.get_by_id('Alpha').subreactions

defaultdict(int,
            {'FusA_mono_elongation': 642.0,
             'Tuf_gtp_regeneration': 642.0,
             'met_addition_at_AUG': 13,
             'ser_addition_at_AGC': 53,
             'ile_addition_at_AUC': 11,
             'gln_addition_at_CAG': 24,
             'thr_addition_at_ACC': 46,
             'val_addition_at_GUU': 37,
             'lys_addition_at_AAA': 21,
             'tyr_addition_at_UAU': 23,
             'arg_addition_at_CGU': 29,
             'gly_addition_at_GGU': 77,
             'his_addition_at_CAU': 12,
             'gly_addition_at_GGC': 20,
             'ile_addition_at_AUU': 25,
             'leu_addition_at_CUG': 36,
             'phe_addition_at_UUU': 14,
             'gln_addition_at_CAA': 7,
             'ala_addition_at_GCA': 42,
             'ser_addition_at_AGU': 10,
             'ser_addition_at_UCA': 3,
             'val_addition_at_GUG': 9,
             'leu_addition_at_UUA': 2,
             'pro_addition_at_CCG': 15,
             'thr_a

List the subreactions generated from the nucleatide sequence that are stored in the ProcessData.

In [16]:
me.process_data.get_by_id('Alpha').subreactions_from_sequence

{'met_addition_at_AUG': 13,
 'ser_addition_at_AGC': 53,
 'ile_addition_at_AUC': 11,
 'gln_addition_at_CAG': 24,
 'thr_addition_at_ACC': 46,
 'val_addition_at_GUU': 37,
 'lys_addition_at_AAA': 21,
 'tyr_addition_at_UAU': 23,
 'arg_addition_at_CGU': 29,
 'gly_addition_at_GGU': 77,
 'his_addition_at_CAU': 12,
 'gly_addition_at_GGC': 20,
 'ile_addition_at_AUU': 25,
 'leu_addition_at_CUG': 36,
 'phe_addition_at_UUU': 14,
 'gln_addition_at_CAA': 7,
 'ala_addition_at_GCA': 42,
 'ser_addition_at_AGU': 10,
 'ser_addition_at_UCA': 3,
 'val_addition_at_GUG': 9,
 'leu_addition_at_UUA': 2,
 'pro_addition_at_CCG': 15,
 'thr_addition_at_ACA': 12,
 'arg_addition_at_CGC': 1,
 'asn_addition_at_AAU': 14,
 'pro_addition_at_CCU': 4,
 'gly_addition_at_GGA': 5,
 'pro_addition_at_CCA': 2,
 'asp_addition_at_GAU': 23,
 'glu_addition_at_GAA': 25,
 'ala_addition_at_GCC': 6,
 'asn_addition_at_AAC': 7,
 'asp_addition_at_GAC': 1,
 'trp_addition_at_UGG': 2,
 'tyr_addition_at_UAC': 2,
 'glu_addition_at_GAG': 1,
 'lys_

### Updating the Alpha translation reaction

In [17]:
translation_subreactions = me.process_data.Alpha.subreactions_from_sequence
print("Before adding tRNA subreaction")
print("------------------------------")
print(me.reactions.translation_Alpha.reaction)
print("")
# Link tranlation_data to subreactions and update
for subreaction, value in translation_subreactions.items():
    me.process_data.Alpha.subreactions[subreaction] = value
me.reactions.translation_Alpha.update(verbose=False)
print("After adding tRNA subreaction")
print("-----------------------------")
print(me.reactions.translation_Alpha.reaction)

Before adding tRNA subreaction
------------------------------
0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_Alpha + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 48 ala__L_c + 30 arg__L_c + 21 asn__L_c + 24 asp__L_c + 2.88274348422497*(mu + 0.3915)/mu atp_c + 31 gln__L_c + 26 glu__L_c + 102 gly_c + 13 his__L_c + 36 ile__L_c + 38 leu__L_c + 23 lys__L_c + 3.67795106755342*(mu + 0.3915)/mu mRNA_biomass + 14 met__L_c + 15 phe__L_c + 21 pro__L_c + 0.00921844765619026*mu + 0.00360902225739849 ribosome + 66 ser__L_c + 58 thr__L_c + 2 trp__L_c + 26 tyr__L_c + 49 val__L_c --> 2.88274348422497*(mu + 0.3915)/mu adp_c + 2.80499314128944*(mu + 0.3915)/mu amp_c + 3.28345679012346*(mu + 0.3915)/mu cmp_c + 2.45810699588477*(mu + 0.3915)/mu gmp_c + 642.0 - 2.88274348422497*(mu + 0.3915)/mu h2o_c + 2.88274348422497*(mu + 0.3915)/mu h_c + 2.88274348422497*(mu + 0.3915)/mu pi_c + protein_Alpha + 66.69648898000001 protein_biomass + 2.990397805212

/source/cobrame/cobrame/core/processdata.py:256 UserWarning: met (generic_tRNA_GAG_glu__L_c) does not have formula


List the reactants associated with the Alpha translation reaction. In this list all the subreactions should be listed.

In [18]:
me.reactions.translation_Alpha.reactants

[<Metabolite met__L_c at 0x7ff1165069b0>,
 <Metabolite h2o_c at 0x7ff117207a90>,
 <Metabolite ser__L_c at 0x7ff116506eb8>,
 <Metabolite ile__L_c at 0x7ff116512208>,
 <Metabolite gln__L_c at 0x7ff116512278>,
 <Metabolite thr__L_c at 0x7ff116506e10>,
 <Metabolite val__L_c at 0x7ff116506ac8>,
 <Metabolite lys__L_c at 0x7ff116506b70>,
 <Metabolite tyr__L_c at 0x7ff116512470>,
 <Metabolite arg__L_c at 0x7ff117359dd8>,
 <Metabolite gly_c at 0x7ff116506cf8>,
 <Metabolite his__L_c at 0x7ff116512390>,
 <Metabolite leu__L_c at 0x7ff116506ba8>,
 <Metabolite phe__L_c at 0x7ff116506cc0>,
 <Metabolite ala__L_c at 0x7ff116506f60>,
 <Metabolite pro__L_c at 0x7ff1165122e8>,
 <Metabolite asn__L_c at 0x7ff116512048>,
 <Metabolite asp__L_c at 0x7ff116512160>,
 <Metabolite glu__L_c at 0x7ff1165120b8>,
 <Metabolite trp__L_c at 0x7ff1165124e0>,
 <Ribosome ribosome at 0x7ff116512550>,
 <TranscribedGene RNA_Alpha at 0x7ff11732fc88>,
 <Metabolite atp_c at 0x7ff1172079b0>,
 <Complex RNA_degradosome at 0x7ff11722

This completes the creation of the translation reaction

## 6. Create Demand Reaction for Alpha

In [19]:
new_reaction = cobrame.MEReaction('DM_protein_Alpha')
me.add_reaction(new_reaction)

In [20]:
me.reactions.get_by_id('DM_protein_Alpha').reaction = 'protein_Alpha ->'
me.reactions.get_by_id('DM_protein_Alpha').reaction

'protein_Alpha --> '

In [21]:
rxn = me.reactions.DM_protein_Alpha
data = rxn
pd.DataFrame({i: str(v) for i, v in data.__dict__.items()}, index=['Attribute Values']).T

,Attribute Values
id,DM_protein_Alpha
name,
notes,{}
annotation,{}
_gene_reaction_rule,
subsystem,
_genes,set()
_metabolites,{<TranslatedGene protein_Alpha at 0x7ff1172f9be0>: -1}
_model,iJL1678b-ME
_objective_coefficient,0.0


## 7. Test the Alpha ME Model

In [ ]:
me.reactions.DM_protein_Alpha.upper_bound = 0.00105
me.reactions.DM_protein_Alpha.lower_bound = 0.00105
solve_me_model(me, 1., min_mu = .1, precision=1e-2, using_soplex=False)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 60.358364 seconds
Finished substituting S,lb,ub in 4.076777 seconds
Finished makeME_LP in 0.771983 seconds
Getting MINOS parameters from ME_NLP...
1 0.5 0.5 1.0 0.5 optimal
Finished substituting S,lb,ub in 4.048906 seconds


## 8. Identify and List the Fluxes Associated with the Alpha Transcription Lumped Reaction

The reaction id of the Alpha transcription reaction is "transcription_TU_Alpha" and requires the "RNAP70-CPLX" metabolite (RNA polymerase) as one of its reactants and produces the mRNA metabolite "RNA_Alpha". The "RNAP70-CPLX" is created by the "formation_RNAP70-CPLX" reaction. The default fluxes through these reaction are:

In [ ]:
print('transcription_TU_Alpha =', me.reactions.get_by_id('transcription_TU_Alpha').x)
print('formation_RNAP70-CPLX =', me.reactions.get_by_id('formation_RNAP70-CPLX').x)

## 9. Identify and List the Fluxes Associated with the Alpha Translation Lumped Reaction

The reaction id of the Alpha translation reaction is "translation_Alpha" and requires the mRNA metabolite "RNA_Alpha" as one of its reactants and produces the protein metabolite ("TranslatedGene") "protein_Alpha". To begin with, let's look at the fluxes of the RNAPs (Alpha uses the "RNAP70-CPLX" RNAP).

In [ ]:
print('formation_CPLX0-221 = ', me.reactions.get_by_id('formation_CPLX0-221').x)
print('formation_CPLX0-222 = ', me.reactions.get_by_id('formation_CPLX0-222').x)
print('formation_RNAPE-CPLX = ', me.reactions.get_by_id('formation_RNAPE-CPLX').x)
print('formation_RNAP32-CPLX = ', me.reactions.get_by_id('formation_RNAP32-CPLX').x)
print('formation_RNAP54-CPLX = ', me.reactions.get_by_id('formation_RNAP54-CPLX').x)
print('formation_RNAP70-CPLX = ', me.reactions.get_by_id('formation_RNAP70-CPLX').x)
print('formation_RNAPS-CPLX = ', me.reactions.get_by_id('formation_RNAPS-CPLX').x)

List the fluxes for the "SummaryVariable" reactions that are associated with the growth-rate ("biomass_dilution").

In [ ]:
print('protein_biomass_to_biomass = ', me.reactions.get_by_id('protein_biomass_to_biomass').x)
print('mRNA_biomass_to_biomass = ', me.reactions.get_by_id('mRNA_biomass_to_biomass').x)
print('tRNA_biomass_to_biomass = ', me.reactions.get_by_id('tRNA_biomass_to_biomass').x)
print('rRNA_biomass_to_biomass = ', me.reactions.get_by_id('rRNA_biomass_to_biomass').x)
print('ncRNA_biomass_to_biomass = ', me.reactions.get_by_id('ncRNA_biomass_to_biomass').x)
print('DNA_biomass_to_biomass = ', me.reactions.get_by_id('DNA_biomass_to_biomass').x)
print('lipid_biomass_to_biomass = ', me.reactions.get_by_id('lipid_biomass_to_biomass').x)
print('constituent_biomass_to_biomass = ', me.reactions.get_by_id('constituent_biomass_to_biomass').x)
print('prosthetic_group_biomass_to_biomass = ', me.reactions.get_by_id('prosthetic_group_biomass_to_biomass').x)
print('peptidoglycan_biomass_to_biomass = ', me.reactions.get_by_id('peptidoglycan_biomass_to_biomass').x)
print('dummy_protein_to_mass = ', me.reactions.get_by_id('dummy_protein_to_mass').x)
print('biomass_constituent_demand = ', me.reactions.get_by_id('biomass_constituent_demand').x)
print('Demand_pe161_p = ', me.reactions.get_by_id('Demand_pe161_p').x)
print('Demand_pe160_p = ', me.reactions.get_by_id('Demand_pe160_p').x)
print('Demand_pe161_p = ', me.reactions.get_by_id('Demand_pe161_p').x)
print('Demand_pe161_c = ', me.reactions.get_by_id('Demand_pe161_c').x)
print('Demand_pe160_c = ', me.reactions.get_by_id('Demand_pe160_c').x)
print('Demand_kdo2lipid4_e = ', me.reactions.get_by_id('Demand_kdo2lipid4_e').x)

### Find the amino acid count

In [ ]:
me.process_data.Alpha.amino_acid_count

## 10. Tracking the tRNA flux

From these amino acid counts it can be seen that glycine and alanine are the two amino acids that dominate the production of Alpha. Let's track the impact of the tRNA production of the reactions that charge the tRNA. 

### Reactions that create charged tRNA metabolites for both Glycine

There are four different codons that can be used to create glycine tRNA. The metabolites associated with these glycine codons are

#### Glycine Charged tRNA Metabolites
1. generic_tRNA_GGC_gly_c
- generic_tRNA_GGU_gly_c
- generic_tRNA_GGG_gly_c
- generic_tRNA_GGA_gly_c

#### Glycine tRNA Charging Reactions
The reactions, based on codons, that create these charged tRNA metabolites are

##### GGC
1. charging_tRNA_b1911_GGC
- charging_tRNA_b4163_GGC
- charging_tRNA_b4164_GGC
- charging_tRNA_b4165_GGC

##### GGU
1.  charging_tRNA_b1911_GGU
- charging_tRNA_b4163_GGU
- charging_tRNA_b4164_GGU
- charging_tRNA_b4165_GGU

##### GGG
1.  charging_tRNA_b2864_GGG

##### GGA
1. charging_tRNA_b3978_GGA

Note that there are six different reactions that create these glycine tRNA metabolites. Also note that there are two reactions that create the glycine tRNA for three of the codons.

### Simulating the growth-rate based on the production of Alpha

In [ ]:
Alpha_protein = [0.0000, 0.0001, 0.0003, 0.0006, 0.0009, 0.001, 0.00105]
growth_rate = []
transcription_rate = []
translation_rate = []
ribosome_rate = []
ribosome_shadow_price = []
acetate_rate = []
ATP_flux = []

# Glycine tRNA
charging_tRNA_b1911_GGC_data = []
charging_tRNA_b4163_GGC_data = []
charging_tRNA_b4164_GGC_data = []
charging_tRNA_b4165_GGC_data = []
charging_tRNA_b1911_GGU_data = []
charging_tRNA_b4163_GGU_data = []
charging_tRNA_b4164_GGU_data = []
charging_tRNA_b4165_GGU_data = []
charging_tRNA_b2864_GGG_data = []
charging_tRNA_b3978_GGA_data = []

# Alanine tRNA
charging_tRNA_b0203_GCA_data = []
charging_tRNA_b3276_GCA_data = []
charging_tRNA_b3853_GCA_data = []
charging_tRNA_b0203_GCG_data = []
charging_tRNA_b3276_GCG_data = []
charging_tRNA_b3853_GCG_data = []
charging_tRNA_b0203_GCU_data = []
charging_tRNA_b3276_GCU_data = []
charging_tRNA_b3853_GCU_data = []
charging_tRNA_b2396_GCC_data = []  
charging_tRNA_b2397_GCC_data = [] 

# Cell Operation
glucose_data = []
oxygen_data = []
acetate_data = []
ammonium_data = []
external_protons_data = []
atp_synthase = []
ATPS4rpp_REV_ATPSYN_CPLX_EG10106_MONOMER_mod_mg2_data = []
ATPS4rpp_FWD_ATPSYN_CPLX_EG10106_MONOMER_mod_mg2_data = []
ATPS4rpp_REV_ATPSYN_CPLX_mod_mg2_data = []
ATPS4rpp_FWD_ATPSYN_CPLX_mod_mg2_data = []

for x in Alpha_protein:
    print('\nAlpha Protein # ',x)
    me.reactions.DM_protein_Alpha.upper_bound = x
    me.reactions.DM_protein_Alpha.lower_bound = x
    solve_me_model(me, 1., min_mu = .1, precision=1e-2, using_soplex=False)
    growth_rate.append(me.solution.f)
    transcription_rate.append(me.reactions.transcription_TU_Alpha.x)
    translation_rate.append(me.reactions.translation_Alpha.x)
    ribosome_rate.append(me.reactions.get_by_id('formation_ribosome').x)
    ribosome_shadow_price.append(me.metabolites.get_by_id('ribosome').y)
    acetate_rate.append(me.reactions.get_by_id('EX_ac_e').x)
    ATP_flux.append(me.reactions.get_by_id('ATPS4rpp_FWD_ATPSYN-CPLX_mod_mg2').x)
    # Glycine tRNA
    charging_tRNA_b1911_GGC_data.append(me.reactions.charging_tRNA_b1911_GGC.x)
    charging_tRNA_b4163_GGC_data.append(me.reactions.charging_tRNA_b4163_GGC.x)
    charging_tRNA_b4164_GGC_data.append(me.reactions.charging_tRNA_b4164_GGC.x)
    charging_tRNA_b4165_GGC_data.append(me.reactions.charging_tRNA_b4165_GGC.x)
    charging_tRNA_b1911_GGU_data.append(me.reactions.charging_tRNA_b1911_GGU.x)
    charging_tRNA_b4163_GGU_data.append(me.reactions.charging_tRNA_b4163_GGU.x)
    charging_tRNA_b4164_GGU_data.append(me.reactions.charging_tRNA_b4164_GGU.x)
    charging_tRNA_b4165_GGU_data.append(me.reactions.charging_tRNA_b4165_GGU.x)
    charging_tRNA_b2864_GGG_data.append(me.reactions.charging_tRNA_b2864_GGG.x)
    charging_tRNA_b3978_GGA_data.append(me.reactions.charging_tRNA_b3978_GGA.x)
    # Alanine tRNA
    charging_tRNA_b0203_GCA_data.append(me.reactions.charging_tRNA_b0203_GCA.x)
    charging_tRNA_b3276_GCA_data.append(me.reactions.charging_tRNA_b3276_GCA.x)
    charging_tRNA_b3853_GCA_data.append(me.reactions.charging_tRNA_b3853_GCA.x)
    charging_tRNA_b0203_GCG_data.append(me.reactions.charging_tRNA_b0203_GCG.x)
    charging_tRNA_b3276_GCG_data.append(me.reactions.charging_tRNA_b3276_GCG.x)
    charging_tRNA_b3853_GCG_data.append(me.reactions.charging_tRNA_b3853_GCG.x)
    charging_tRNA_b0203_GCU_data.append(me.reactions.charging_tRNA_b0203_GCU.x)
    charging_tRNA_b3276_GCU_data.append(me.reactions.charging_tRNA_b3276_GCU.x)
    charging_tRNA_b3853_GCU_data.append(me.reactions.charging_tRNA_b3853_GCU.x)
    charging_tRNA_b2396_GCC_data.append(me.reactions.charging_tRNA_b2396_GCC.x)  
    charging_tRNA_b2397_GCC_data.append(me.reactions.charging_tRNA_b2397_GCC.x) 
    # Cell operation
    glucose_data.append(me.reactions.get_by_id('EX_glc__D_e').x)
    oxygen_data.append(me.reactions.get_by_id('EX_o2_e').x)
    acetate_data.append(me.reactions.get_by_id('EX_ac_e').x)
    ammonium_data.append(me.reactions.get_by_id('EX_nh4_e').x)
    external_protons_data.append(me.reactions.get_by_id('EX_h_e').x)
    ATPS4rpp_REV_ATPSYN_CPLX_EG10106_MONOMER_mod_mg2_data.append(me.reactions.get_by_id('ATPS4rpp_REV_ATPSYN-CPLX_EG10106-MONOMER_mod_mg2').x)
    ATPS4rpp_FWD_ATPSYN_CPLX_EG10106_MONOMER_mod_mg2_data.append(me.reactions.get_by_id('ATPS4rpp_FWD_ATPSYN-CPLX_EG10106-MONOMER_mod_mg2').x)
    ATPS4rpp_REV_ATPSYN_CPLX_mod_mg2_data.append(me.reactions.get_by_id('ATPS4rpp_REV_ATPSYN-CPLX_mod_mg2').x)
    ATPS4rpp_FWD_ATPSYN_CPLX_mod_mg2_data.append(me.reactions.get_by_id('ATPS4rpp_FWD_ATPSYN-CPLX_mod_mg2').x)
    #atp_synthase.append(me.reactions.get_by_id(me.reactions.get_by_id('ATPS4rpp_FWD_ATPSYN-CPLX_EG10106-MONOMER_mod_mg2').x)


print('Growth-rate =', growth_rate)
print('Transcription Rate =', transcription_rate)
print('Translation Rate =', translation_rate)

In [ ]:
import matplotlib.pyplot as plt
  
# x axis values
x = Alpha_protein
# corresponding y axis values

plt.figure(figsize=(7,5))

# plotting the points 
plt.plot(x, glucose_data, label = "glucose_data")
plt.plot(x, oxygen_data, label = "oxygen_data")

# naming the x axis
plt.xlabel('Alpha Protein Flux')
# naming the y axis
plt.ylabel('Cellular Flux Values')
  
# giving a title to my graph
plt.title('Glucose and Oxygen Flux Values vs Alpha Protein Flux Production')
  
# function to show the plot
plt.legend()
#plt.yscale('log')
plt.show()

In [ ]:
glucose_data

In [ ]:
oxygen_data

In [ ]:
import matplotlib.pyplot as plt
  
# x axis values
x = Alpha_protein
# corresponding y axis values

plt.figure(figsize=(12,12))

# plotting the points 
#plt.plot(x, growth_rate, label = "growth_rate")
plt.plot(x, glucose_data, label = "glucose_data")
plt.plot(x, oxygen_data, label = "oxygen_data")
plt.plot(x, acetate_data, label = "acetate_data")
plt.plot(x, ammonium_data, label = "ammonium_data")

plt.plot(x, external_protons_data, label = "external_protons_data")
plt.plot(x, ATPS4rpp_REV_ATPSYN_CPLX_EG10106_MONOMER_mod_mg2_data, label = "ATPS4rpp_REV_ATPSYN_CPLX_EG10106_MONOMER_mod_mg2_data")
plt.plot(x, ATPS4rpp_FWD_ATPSYN_CPLX_EG10106_MONOMER_mod_mg2_data, label = "ATPS4rpp_FWD_ATPSYN_CPLX_EG10106_MONOMER_mod_mg2_data")
plt.plot(x, ATPS4rpp_REV_ATPSYN_CPLX_mod_mg2_data, label = "ATPS4rpp_REV_ATPSYN_CPLX_mod_mg2_data")
plt.plot(x, ATPS4rpp_FWD_ATPSYN_CPLX_mod_mg2_data, label = "ATPS4rpp_FWD_ATPSYN_CPLX_mod_mg2_data")

# naming the x axis
plt.xlabel('Alpha_protein')
# naming the y axis
plt.ylabel('Cellular Flux Values')
  
# giving a title to my graph
plt.title('Key Cellular Flux Values vs Alpha Protein Production')
  
# function to show the plot
plt.legend()
#plt.yscale('log')
plt.show()

### Plot the growth-rate with respect to Alpha production

#### Plotting the flux of the individual tRNA charging reactions

In [ ]:
import matplotlib.pyplot as plt
  
# x axis values
x = Alpha_protein
# corresponding y axis values

plt.figure(figsize=(12,12))

# plotting the points 
#plt.plot(x, growth_rate, label = "growth_rate")
plt.plot(x, charging_tRNA_b1911_GGC_data, label = "charging_tRNA_b1911_GGC_data")
plt.plot(x, charging_tRNA_b4163_GGC_data, label = "charging_tRNA_b4163_GGC_data")
plt.plot(x, charging_tRNA_b4164_GGC_data, label = "charging_tRNA_b4164_GGC_data")
plt.plot(x, charging_tRNA_b4165_GGC_data, label = "charging_tRNA_b4165_GGC_data")

plt.plot(x, charging_tRNA_b1911_GGU_data, label = "charging_tRNA_b1911_GGU_data")
plt.plot(x, charging_tRNA_b4163_GGU_data, label = "charging_tRNA_b4163_GGU_data")
plt.plot(x, charging_tRNA_b4164_GGU_data, label = "charging_tRNA_b4164_GGU_data")
plt.plot(x, charging_tRNA_b4165_GGU_data, label = "charging_tRNA_b4165_GGU_data")

plt.plot(x, charging_tRNA_b2864_GGG_data, label = "charging_tRNA_b2864_GGG_data")
plt.plot(x, charging_tRNA_b3978_GGA_data, label = "charging_tRNA_b3978_GGA_data")

# naming the x axis
plt.xlabel('Alpha_protein')
# naming the y axis
plt.ylabel('Glycine tRNA Flux Values')
  
# giving a title to my graph
plt.title('Glycine tRNA Flux Values vs Alpha Protein Production')
  
# function to show the plot
plt.legend()
#plt.yscale('log')
plt.show()

#### Plot the growth-rate with respect to the codon-related flux

In [ ]:
import numpy as np

GGC = [charging_tRNA_b1911_GGC_data, charging_tRNA_b4163_GGC_data, charging_tRNA_b4164_GGC_data, 
          charging_tRNA_b4165_GGC_data]
GGC = sum(map(np.array, GGC))
print('GGC = ', GGC)

GGU = [charging_tRNA_b1911_GGU_data, charging_tRNA_b4163_GGU_data, charging_tRNA_b4164_GGU_data,  
       charging_tRNA_b4165_GGU_data]
GGU = sum(map(np.array, GGU))
print('GGU = ', GGU)

GGG = charging_tRNA_b2864_GGG_data
print('GGG = ', GGG)

GGA = charging_tRNA_b3978_GGA_data
print('GGA = ', GGA)

# x axis values
x = Alpha_protein
# corresponding y axis values

plt.figure(figsize=(7,5))

# plotting the points 
plt.plot(x, GGC, label = "GGC")
plt.plot(x, GGU, label = "GGU")
plt.plot(x, GGG, label = "GGG")
plt.plot(x, GGA, label = "GGA")

# naming the x axis
plt.xlabel('Alpha_protein')
# naming the y axis
plt.ylabel('Glycine Total tRNA Codon Flux Values')
  
# giving a title to my graph
plt.title('Glycine Total tRNA Codon Flux Values vs Alpha Protein Production')
  
# function to show the plot
plt.legend()
#plt.yscale('log')
plt.show()

#### Plot the growth-rate with respect to the charging reaction flux

In [ ]:
import numpy as np

b1911_data = [charging_tRNA_b1911_GGC_data, charging_tRNA_b1911_GGU_data]
b1911_data = sum(map(np.array, b1911_data))
print('b1911_data = ', b1911_data)

b4163_data = [charging_tRNA_b4163_GGC_data, charging_tRNA_b4163_GGU_data]
b4163_data = sum(map(np.array, b4163_data))
print('b4163_data = ', b4163_data)

b4164_data = [charging_tRNA_b4164_GGC_data, charging_tRNA_b4164_GGU_data]
b4164_data = sum(map(np.array, b4164_data))
print('b4164_data = ', b4164_data)

b4165_data = [charging_tRNA_b4165_GGC_data, charging_tRNA_b4165_GGU_data]
b4165_data = sum(map(np.array, b4165_data))
print('b4165_data = ', b4165_data)

b2864_data = charging_tRNA_b2864_GGG_data
print('b2864_data = ', b2864_data)

b3978_data = charging_tRNA_b3978_GGA_data
print('b3978_data = ', b3978_data)

# x axis values
x = Alpha_protein
# corresponding y axis values

plt.figure(figsize=(7,5))

# plotting the points 
plt.plot(x, b1911_data, label = "b1911_data")
plt.plot(x, b4163_data, label = "b4163_data")
plt.plot(x, b4164_data, label = "b4164_data")
plt.plot(x, b4165_data, label = "b4165_data")
plt.plot(x, b2864_data, label = "b2864_data")
plt.plot(x, b3978_data, label = "b3978_data")

# naming the x axis
plt.xlabel('Alpha_protein')
# naming the y axis
plt.ylabel('Total Gene tRNA Flux Values')
  
# giving a title to my graph
plt.title('Total Gene tRNA Flux Values vs Alpha Protein Production')
  
# function to show the plot
plt.legend()
#plt.yscale('log')
plt.show()

### Reactions that create charged tRNA metabolites for both Alanine

There are four different codons that can be used to create alanine tRNA. The metabolites associated with these alanine codons are

### Alanine tRNA Charged Metabolites
1. generic_tRNA_GCA_ala__L_c
- generic_tRNA_GCG_ala__L_c
- generic_tRNA_GCU_ala__L_c
- generic_tRNA_GCC_ala__L_c

### Alanine tRNA Charging Reactions
The reactions, based on codons, that create these charged tRNA metabolites are

##### GCA
1. charging_tRNA_b0203_GCA
- charging_tRNA_b3276_GCA
- charging_tRNA_b3853_GCA

##### GCG
1. charging_tRNA_b0203_GCG
- charging_tRNA_b3276_GCG
- charging_tRNA_b3853_GCG

##### GCU
1. charging_tRNA_b0203_GCU
- charging_tRNA_b3276_GCU
- charging_tRNA_b3853_GCU

##### GCC
1. charging_tRNA_b2396_GCC
- charging_tRNA_b2397_GCC

Note that there are five different reactions that create these alanine tRNA metabolites. Also note that there are three reactions that create the alanine tRNA for three of the codons.

#### Plotting the flux of the individual alanine tRNA charging reactions

In [ ]:
import matplotlib.pyplot as plt
  
# x axis values
x = Alpha_protein
# corresponding y axis values

plt.figure(figsize=(12,12))

# plotting the points 
#plt.plot(x, growth_rate, label = "growth_rate")
plt.plot(x, charging_tRNA_b0203_GCA_data, label = "charging_tRNA_b0203_GCA_data")
plt.plot(x, charging_tRNA_b3276_GCA_data, label = "charging_tRNA_b3276_GCA_data")
plt.plot(x, charging_tRNA_b3853_GCA_data, label = "charging_tRNA_b3853_GCA_data")

plt.plot(x, charging_tRNA_b0203_GCG_data, label = "charging_tRNA_b0203_GCG_data")
plt.plot(x, charging_tRNA_b3276_GCG_data, label = "charging_tRNA_b3276_GCG_data")
plt.plot(x, charging_tRNA_b3853_GCG_data, label = "charging_tRNA_b3853_GCG_data")

plt.plot(x, charging_tRNA_b0203_GCU_data, label = "charging_tRNA_b0203_GCU_data")
plt.plot(x, charging_tRNA_b3276_GCU_data, label = "charging_tRNA_b3276_GCU_data")
plt.plot(x, charging_tRNA_b3853_GCU_data, label = "charging_tRNA_b3853_GCU_data")

plt.plot(x, charging_tRNA_b2396_GCC_data, label = "charging_tRNA_b2396_GCC_data")
plt.plot(x, charging_tRNA_b2397_GCC_data, label = "charging_tRNA_b2397_GCC_data")

# naming the x axis
plt.xlabel('Alpha_protein')
# naming the y axis
plt.ylabel('Alanine tRNA Flux Values')
  
# giving a title to my graph
plt.title('Alanine tRNA Flux Values vs Alpha Protein Production')
  
# function to show the plot
plt.legend()
#plt.yscale('log')
plt.show()

#### Plot the growth-rate with respect to the codon-related flux

In [ ]:
# x axis values
x = Alpha_protein
# corresponding y axis values

plt.figure(figsize=(12,8))

GCA = [charging_tRNA_b0203_GCA_data, charging_tRNA_b3276_GCA_data, charging_tRNA_b3853_GCA_data]
GCA = sum(map(np.array, GCA))
print('GCA = ', GCA)

GCG = [charging_tRNA_b0203_GCG_data, charging_tRNA_b3276_GCG_data, charging_tRNA_b3853_GCG_data]
GCG = sum(map(np.array, GCG))
print('GCG = ', GCG)

GCU = [charging_tRNA_b0203_GCU_data, charging_tRNA_b3276_GCU_data, charging_tRNA_b3853_GCU_data]
GCU = sum(map(np.array, GCU))
print('GCU = ', GCU)

GCC = [charging_tRNA_b2396_GCC_data, charging_tRNA_b2397_GCC_data]
GCC = sum(map(np.array, GCC))
print('GCC = ', GCC)

total = [GCA, GCG, GCU, GCC]
total = sum(map(np.array, total))
print('total = ', total)

# plotting the points 
#plt.plot(x, growth_rate, label = "growth_rate")
plt.plot(x, GCA, label = "GCA")
plt.plot(x, GCG, label = "GCG")
plt.plot(x, GCU, label = "GCU")
plt.plot(x, GCC, label = "GCC")
#plt.plot(x, total, label = "Total")

# naming the x axis
plt.xlabel('Alpha_protein')
# naming the y axis
plt.ylabel('Alanine tRNA Codon Flux Values')
  
# giving a title to my graph
plt.title('Alanine tRNA Codon Flux Values vs Alpha Protein Production')
  
# function to show the plot
plt.legend()
#plt.yscale('log')
plt.show()

#### Plot the growth-rate with respect to the reaction flux

In [ ]:
# x axis values
x = Alpha_protein
# corresponding y axis values

plt.figure(figsize=(12,8))

b0203 = [charging_tRNA_b0203_GCA_data, charging_tRNA_b0203_GCG_data, charging_tRNA_b0203_GCU_data]
b0203 = sum(map(np.array, b0203))
print('b0203 = ', b0203)

b3276 = [charging_tRNA_b3276_GCA_data, charging_tRNA_b3276_GCG_data, charging_tRNA_b3276_GCU_data]
b3276 = sum(map(np.array, b3276))
print('b3276 = ', b3276)

b3853 = [charging_tRNA_b3853_GCA_data, charging_tRNA_b3853_GCG_data, charging_tRNA_b3853_GCU_data]
b3853 = sum(map(np.array, b3853))
print('b3853 = ', b3853)

b2396 = [charging_tRNA_b2396_GCC_data]
b2396 = sum(map(np.array, b2396))
print('b2396 = ', b2396)

b2397 = [charging_tRNA_b2397_GCC_data]
b2397 = sum(map(np.array, b2397))
print('b2397 = ', b2397)

total = [b0203, b3276, b3853, b2396,b2397]
total = sum(map(np.array, total))
print('total = ', total)

# plotting the points 
#plt.plot(x, growth_rate, label = "growth_rate")
plt.plot(x, b0203, label = "b0203")
plt.plot(x, b3276, label = "b3276")
plt.plot(x, b3853, label = "b3853")
plt.plot(x, b2396, label = "b2396")
plt.plot(x, b2397, label = "b2397")
#plt.plot(x, total, label = "total")

# naming the x axis
plt.xlabel('Alpha_protein')
# naming the y axis
plt.ylabel('Alanine tRNA Gene Flux Values')
  
# giving a title to my graph
plt.title('Alanine tRNA Gene Flux Values vs Alpha Protein Production')
  
# function to show the plot
plt.legend()
#plt.yscale('log')
plt.show()

#### Plot the transcription, translation, and ribosome flux with respect to Alpha production

In [ ]:
import matplotlib.pyplot as plt
  
# x axis values
x = Alpha_protein
# corresponding y axis values

plt.figure(figsize=(12,12))

# plotting the points 
#plt.plot(x, growth_rate, label = "growth_rate")
plt.plot(x, transcription_rate, label = "transcription_rate")
plt.plot(x, translation_rate, label = "translation_rate")
plt.plot(x, ribosome_rate, label = "ribosome_rate")
#plt.plot(x, ribosome_shadow_price, label = "ribosome_shadow_price")

# naming the x axis
plt.xlabel('Alpha_protein')
# naming the y axis
plt.ylabel('Flux Values')
  
# giving a title to my graph
plt.title('Key Data vs Alpha Protein Production')
  
# function to show the plot
plt.legend()
plt.yscale('log')
plt.show()

#### Plot the growth-rate with respect to Alpha production

In [ ]:
import matplotlib.pyplot as plt
  
# x axis values
x = Alpha_protein
# corresponding y axis values
y = growth_rate
  
# plotting the points 
plt.plot(x, y)
  
# naming the x axis
plt.xlabel('Alpha Protein')
# naming the y axis
plt.ylabel('Growth-rate')
  
# giving a title to my graph
plt.title('Cell Growth-rate vs Alpha Protein')
  
# function to show the plot
plt.show()

#### Plot the transcription flux with respect to Alpha production

In [ ]:
# x axis values
x = Alpha_protein
# corresponding y axis values
y = transcription_rate
  
# plotting the points 
plt.plot(x, y)
  
# naming the x axis
plt.xlabel('Alpha Protein')
# naming the y axis
plt.ylabel('Transcription Rate')
  
# giving a title to my graph
plt.title('Cell Transcription Rate vs Alpha Protein')
  
# function to show the plot
plt.show()

#### Plot the translation flux with respect to Alpha production

In [ ]:
# x axis values
x = Alpha_protein
# corresponding y axis values
y = translation_rate
  
# plotting the points 
plt.plot(x, y)
  
# naming the x axis
plt.xlabel('Alpha Protein')
# naming the y axis
plt.ylabel('Translation Rate')
  
# giving a title to my graph
plt.title('Cell Translation Rate vs Alpha Protein')
  
# function to show the plot
plt.show()

#### Plot the ribosome flux with respect to Alpha production

In [ ]:
# x axis values
x = Alpha_protein
# corresponding y axis values
y = ribosome_rate
  
# plotting the points 
plt.plot(x, y)
  
# naming the x axis
plt.xlabel('Alpha Protein')
# naming the y axis
plt.ylabel('Ribosome Flux')
  
# giving a title to my graph
plt.title('Ribosome Flux vs Alpha Protein')
  
# function to show the plot
plt.show()

#### Plot the ribosome shadow prices with respect to Alpha production

In [ ]:
# x axis values
x = Alpha_protein
# corresponding y axis values
y = ribosome_shadow_price
  
# plotting the points 
plt.plot(x, y)
  
# naming the x axis
plt.xlabel('Alpha Protein')
# naming the y axis
plt.ylabel('Ribosome Shadow Price')
  
# giving a title to my graph
plt.title('Ribosome Shadow Price vs Alpha Protein')
  
# function to show the plot
plt.show()

#### Plot the secreted acetate flux with respect to Alpha production

In [ ]:
# x axis values
x = Alpha_protein
# corresponding y axis values
y = acetate_rate
  
# plotting the points 
plt.plot(x, y)
  
# naming the x axis
plt.xlabel('Alpha Protein')
# naming the y axis
plt.ylabel('Acetate Flux Rate')
  
# giving a title to my graph
plt.title('Acetate Flux Rate vs Alpha Protein')
  
# function to show the plot
plt.show()

#### Plot the ATP production flux with respect to Alpha production

In [ ]:
# x axis values
x = Alpha_protein
# corresponding y axis values
y = ATP_flux
  
# plotting the points 
plt.plot(x, y)
  
# naming the x axis
plt.xlabel('Alpha Protein')
# naming the y axis
plt.ylabel('ATP Flux Rate')
  
# giving a title to my graph
plt.title('ATP Flux Rate vs Alpha Protein')
  
# function to show the plot
plt.show()

In [ ]:
show_escher_map(me).display_in_notebook()

In [ ]:
print('transcription_TU_Alpha =', me.reactions.get_by_id('transcription_TU_Alpha').x)
print('formation_RNAP70-CPLX =', me.reactions.get_by_id('formation_RNAP70-CPLX').x)

In [ ]:
me.metabolites.protein_Alpha.reactions

In [ ]:
me.reactions.translation_Alpha.x